In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')

import os


In [2]:
# change to own path!
path = r"/home/dazai/Documents/Process mining/Data/printer data/"
df_train = pd.read_csv(f'{path}/event-log-training.csv')
df_test = pd.read_csv(f'{path}/event-log-test.csv')


# cleaning
df_data = pd.concat([df_train, df_test])
df_data['event time:timestamp'] = pd.to_datetime(df_data['event time:timestamp'], dayfirst=True)
df_data = df_data.sort_values(by=['event time:timestamp'])
df_data.reset_index(inplace=True, drop=True)
df_data.drop(['case description', 'event org:resource'], axis=1, inplace=True)

# remove whitespace at beginning and end of column name
df_data.columns = df_data.columns.str.strip()


## splitting data

In [32]:
def split(df_data, percent):
    """input percentage of testing data to split"""
    index = round(df_data.shape[0]*(percent/100))
    
    testset = df_data.iloc[index:]
    testset.reset_index(drop=True, inplace=True)
    return df_data.iloc[:index], testset

In [20]:
# def split(df_data, percent):
#     """input percentage of training data to split
#        if a job starts before split it wil be excluded"""
#     index = round(df_data.shape[0]*(percent/100))
    
#     train = df_data.iloc[:index]
#     test = df_data.iloc[index:]
#     # display(test)


#     # look through cases to check for completeness
#     for i in train['case concept:name'].unique():
#         case = train[train['case concept:name']==i]
#         name = case['event concept:name'].iloc[0]

#         # if there is no start and end (2) then remove case
#         if len(case[case['event concept:name']==name]) != 2:
#             train = train[train['case concept:name']!=i]
#             test = test[test['case concept:name']!=i]


#     return train, test


,eventID,case concept:name,case Class,event concept:name,event lifecycle:transition,event time:timestamp
0,0,1,Print,Job,start,1970-01-01 01:00:00
1,1,1,Print,Remote Print,complete,1970-01-01 01:15:00
2,2,1,Print,Read Print Options,complete,1970-01-01 01:26:00
3,3,1,Print,Rasterization,start,1970-01-01 01:38:00
4,4,1,Print,Interpretation,start,1970-01-01 01:51:00
...,...,...,...,...,...,...
32791,240518168879,6,Print,Transfer Toner (drum to paper),complete,1970-10-15 18:27:00
32792,240518168880,6,Print,Paper Roller Spin Stop,complete,1970-10-15 18:44:00
32793,240518168881,6,Print,Drum Spin Stop,complete,1970-10-15 19:04:00
32794,240518168882,6,Print,Fusing,start,1970-10-15 19:24:00


In [42]:
df_train = split(df_data, 80)[0]

df_test = split(df_data, 80)[1]

# Baseline model next event

In [47]:
# assign column names to variables for ease

cEvent = 'event concept:name'
cNew = 'next event'
cPred = 'predicted next event'

df_train[cNew] = df_train[cEvent].shift(-1)
df_train[cPred] = np.nan
df_train.head()

/tmp/ipykernel_168968/1030834382.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[cNew] = df_train[cEvent].shift(-1)
/tmp/ipykernel_168968/1030834382.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[cPred] = np.nan


,eventID,case concept:name,case Class,event concept:name,event lifecycle:transition,event time:timestamp,next event,predicted next event
0,0,1,Print,Job,start,1970-01-01 01:00:00,Remote Print,NaN
1,1,1,Print,Remote Print,complete,1970-01-01 01:15:00,Read Print Options,NaN
2,2,1,Print,Read Print Options,complete,1970-01-01 01:26:00,Rasterization,NaN
3,3,1,Print,Rasterization,start,1970-01-01 01:38:00,Interpretation,NaN
4,4,1,Print,Interpretation,start,1970-01-01 01:51:00,Unformatted Text,NaN


In [48]:
lstEvents = df_train[cEvent].unique().tolist()

# helper function
def simplePred(df,lst, cEvnt, cNw, cPrd):
    '''loops through the list of events and adds them to the df'''
    i = 0
    while i < len(lst):
        df.loc[df[cEvnt] == lst[i], cPrd] = df[df[cEvnt] == lst[i]][cNw].mode()[0]
        i += 1
    return df

def simpleAccuracy(df, cReal, cPrd):
    '''computes a simple accuracy score'''
    return len(df[df[cReal] == df[cPred]]) / len(df)

In [49]:
simplePred(df_train, lstEvents, cEvent, cNew, cPred)

/home/dazai/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,eventID,case concept:name,case Class,event concept:name,event lifecycle:transition,event time:timestamp,next event,predicted next event
0,0,1,Print,Job,start,1970-01-01 01:00:00,Remote Print,Copy/Scan
1,1,1,Print,Remote Print,complete,1970-01-01 01:15:00,Read Print Options,Read Print Options
2,2,1,Print,Read Print Options,complete,1970-01-01 01:26:00,Rasterization,Rasterization
3,3,1,Print,Rasterization,start,1970-01-01 01:38:00,Interpretation,Interpretation
4,4,1,Print,Interpretation,start,1970-01-01 01:51:00,Unformatted Text,Screening
...,...,...,...,...,...,...,...,...
32791,240518168879,6,Print,Transfer Toner (drum to paper),complete,1970-10-15 18:27:00,Paper Roller Spin Stop,Paper Roller Spin Stop
32792,240518168880,6,Print,Paper Roller Spin Stop,complete,1970-10-15 18:44:00,Drum Spin Stop,Drum Spin Stop
32793,240518168881,6,Print,Drum Spin Stop,complete,1970-10-15 19:04:00,Fusing,Drum Spin Start
32794,240518168882,6,Print,Fusing,start,1970-10-15 19:24:00,Pressure Roller Spin Start,Wipe Toner on Drum


In [50]:
simpleAccuracy(df_train, cNew, cPred)

0.5893401634345652